In [ ]:
'''For testing our network'''
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from psutil import cpu_count
from torchvision import transforms
from torch.utils.data import DataLoader
from dataloader import PicklebotDataset, custom_collate
from mobilenet import MobileNetLarge2D, MobileNetSmall2D, MobileNetSmall3D,MobileNetLarge3D
from movinet import MoViNetA2
from helpers import calculate_accuracy, calculate_accuracy_bce

torch.manual_seed(1234)

def forward_pass(loader):
    model.eval()
    test_losses = [] 
    test_samples = 0
    test_correct = 0

    #calculate the loss
    for test_features,test_labels in tqdm(loader):
        test_features = test_features.to(torch.bfloat16).to(device)
        test_labels = (test_labels.long()).to(device) 
        test_outputs = model(test_features)
        # test_labels = test_labels.unsqueeze(1)
        test_loss = criterion(test_outputs,test_labels)
        test_losses.append(test_loss.item())
        
        test_correct += calculate_accuracy(test_outputs,test_labels)
        
        test_samples += len(test_labels)
        

    return test_losses, test_correct, test_samples


@torch.no_grad()
def estimate_loss():
    #evaluate the model

    

    balls_losses, balls_correct, balls_samples = forward_pass(ball_loader)
    strikes_losses,strikes_correct, strikes_samples = forward_pass(strike_loader)
    test_losses = balls_losses + strikes_losses
    avg_test_loss = np.mean(test_losses)
    print(balls_correct)
    print(strikes_correct)
    balls_accuracy = balls_correct / balls_samples 
    strikes_accuracy = strikes_correct / strikes_samples 
    return avg_test_loss, balls_accuracy, strikes_accuracy 


device = 'cuda' if torch.cuda.is_available() else 'cpu'
std = (0.2104, 0.1986, 0.1829)
mean = (0.3939, 0.3817, 0.3314)
batch_size = 4 
#annotations paths
strike_annotations_file = '/home/henry/Documents/PythonProjects/picklebotdataset/strike_test_labels.csv'
ball_annotations_file = '/home/henry/Documents/PythonProjects/picklebotdataset/ball_test_labels.csv'

#video paths
test_video_paths = '/home/henry/Documents/PythonProjects/picklebotdataset/test_all_together'

#establish our normalization using transforms, 
#note that we are doing this in our dataloader as opposed to in the training loop like with dali
transform = transforms.Normalize(mean,std)


ball_dataset = PicklebotDataset(ball_annotations_file,test_video_paths,transform=transform)
strike_dataset = PicklebotDataset(strike_annotations_file,test_video_paths,transform=transform)
ball_loader = DataLoader(ball_dataset, batch_size=batch_size,shuffle=True,collate_fn=custom_collate,num_workers=cpu_count())
strike_loader = DataLoader(strike_dataset, batch_size=batch_size,shuffle=True,collate_fn=custom_collate,num_workers=cpu_count())


model = MobileNetLarge3D(num_classes=2)
criterion = nn.CrossEntropyLoss()
for i in range(30,63): 
    model.to(device)
    state_dict = torch.load(f'/home/henry/Documents/PythonProjects/old_checkpoints/MobileNetLargeWeighted/MobileNetLarge3D{i}.pth')
    for key in list(state_dict.keys()):
        if key.startswith("_orig_mod."):
            new_key = key.replace("_orig_mod.", "")
            state_dict[new_key] = state_dict[key]
            del state_dict[key]
    model.load_state_dict(state_dict)
    model.to(torch.bfloat16)
    avg_test_loss,balls_accuracy,strikes_accuracy = estimate_loss()

    print(f'mobilenet {i} test loss: {avg_test_loss:.4f}, mobilenet ball test accuracy: {balls_accuracy * 100:.2f}% mobilenet strike test accuracy: {strikes_accuracy * 100:.2f}%')

In [ ]:
#here, we test the ensemble of 48 and 36. 

In [42]:
'''Calculate the number of parameters in each model, for comparison purposes. 
   Note that movinet is about 2.8x larger than mobilenet small, and mobilenet large is about 2.5x larger than mobilenet small.'''

from movinet import MoViNetA2
from mobilenet import MobileNetLarge3D
movinet = MoViNetA2()
mobilenet_large = MobileNetLarge3D()
mobilenet_small = MobileNetSmall3D()

n
movinet_params = sum(p.numel() for p in movinet.parameters())
mobilenet_large_params = sum(p.numel() for p in mobilenet_large.parameters())
mobilenet_small_params = sum(p.numel() for p in mobilenet_small.parameters())
print(f"number of parameters in movinet: {movinet_params}")
print(f"number of parameters in mobilenet large: {mobilenet_large_params}")
print(f"number of parameters in mobilenet small: {mobilenet_small_params}")

number of parameters in movinet: 4660762
number of parameters in mobilenet large: 4191584
number of parameters in mobilenet small: 1672816
